In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'/Users/mananpatel/Documents/vscode/nlp/Text-Summarizer'

In [4]:
#entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class DataTransformationConfig:
  root_dir: Path
  data_path: Path
  tokenizer_name: Path

In [5]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [6]:
#configuration manager for src
class ConfigurationManager:
  def __init__(self,
                config_file_path = CONFIG_FILE_PATH,
                params_file_path = PARAMS_FILE_PATH):
    
    self.config = read_yaml(config_file_path)
    self.params = read_yaml(params_file_path)
    create_directories([self.config.artifacts_root])

  def get_data_transformation_config(self) -> DataTransformationConfig:
    config = self.config.data_transformation

    create_directories([config.root_dir])

    data_transformation_config = DataTransformationConfig(
      root_dir = config.root_dir,
      data_path = config.data_path,
      tokenizer_name = config.tokenizer_name
    )

    return data_transformation_config

In [7]:
#create components
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

[2024-02-20 01:34:02,819]: INFO: utils: NumExpr defaulting to 8 threads.]


In [8]:
class DataTransformation:
  def __init__(self, config: DataTransformationConfig):
    self.config = config
    self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)


  def convert_examples_to_features(self, example_batch):
    input_encodings = self.tokenizer(example_batch['dialogue'], max_length = 1024, truncation = True, )

    with self.tokenizer.as_target_tokenizer():
      target_encodings = self.tokenizer(example_batch['summary'], max_length = 1024, truncation = True)

    return {
      'input_ids': input_encodings['input_ids'],
      'attention_mask': input_encodings['attention_mask'],
      'labels': target_encodings['input_ids'],
    }
  
  def convert(self):
    dataset_samsum = load_from_disk(self.config.data_path)
    dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
    dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

In [9]:
#create pipeline
try:
  config = ConfigurationManager()
  data_transformation_config = config.get_data_transformation_config()
  data_transformation = DataTransformation(config = data_transformation_config)
  data_transformation.convert()

except Exception as e:
  raise e 

[2024-02-20 01:34:03,275]: INFO: common: yaml file : <_io.TextIOWrapper name='config/config.yaml' mode='r' encoding='UTF-8'> loaded successfully]
[2024-02-20 01:34:03,276]: INFO: common: yaml file : <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfully]
[2024-02-20 01:34:03,277]: INFO: common: Created directory: artifacts]
[2024-02-20 01:34:03,277]: INFO: common: Created directory: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/Users/mananpatel/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14732 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/819 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/818 [00:00<?, ? examples/s]